In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from string import digits
from collections import Counter
from pyvi import ViTokenizer
from gensim.models.word2vec import Word2Vec
from keras.utils.np_utils import to_categorical
import re
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data_train = pd.read_csv("train.csv")
data_test = pd.read_csv("test.csv")

In [3]:
labels=data_train.iloc[:, :-1].values
reviews = data_train.iloc[:, -1].values

In [4]:
encoded_labels = []
for label in labels:
    if label == -1:
        encoded_labels.append([1,0,0])
    else:
        encoded_labels.append([0,0,1])
encoded_labels = np.array(encoded_labels)

In [5]:
reviews_processed = []
unlabeled_processed =[]
for review in reviews:
    review_cool_one = ''.join([char for char in review if char not in digits])
    reviews_processed.append(review_cool_one)
print([reviews[0]])
print([reviews_processed[0]])

['Game hay thế này mà anh em không chơi thì phí phạm cuộc đời :33333']
['Game hay thế này mà anh em không chơi thì phí phạm cuộc đời :']


In [6]:
word_reviews = []
all_words = []
for review in reviews_processed:
    review = ViTokenizer.tokenize(review.lower())
    word_reviews.append(review.split())

print (word_reviews[0])

['game', 'hay', 'thế', 'này', 'mà', 'anh_em', 'không', 'chơi', 'thì', 'phí_phạm', 'cuộc_đời', ':']


In [7]:
EMBEDDING_DIM = 400 # HOW BIG IS EACH WORD VECTOR
MAX_VOCAB_SIZE = 10000 # HOW MANY UNIQUE WORDS TO USE
MAX_SEQUENCE_LENGTH = 300 # MAX NUMBER OF WORDS IN A COMMENT TO USE

In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer  

In [9]:
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, lower=True, char_level=False)
tokenizer.fit_on_texts(word_reviews)
sequences_train = tokenizer.texts_to_sequences(word_reviews)
word_index = tokenizer.word_index

In [10]:
data = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH)
labels = encoded_labels

In [11]:
print("Shape of X_train and X validation tensor:", data.shape)
print("Shape of label train and validation tensor:", labels.shape)

Shape of X_train and X validation tensor: (1896, 300)
Shape of label train and validation tensor: (1896, 3)


In [15]:
labels_test=data_test.iloc[:, :-1].values
reviews_test = data_test.iloc[:, -1].values

In [16]:
encoded_labels_test = []
for label_test in labels_test:
    if label_test == -1:
        encoded_labels_test.append([1,0,0])
    else:
        encoded_labels_test.append([0,0,1])
encoded_labels_test = np.array(encoded_labels_test)

In [17]:
reviews_processed_test = []
unlabeled_processed_test =[]
for review_test in reviews_test:
    review_cool_one = ''.join([char for char in review_test if char not in digits])
    reviews_processed_test.append(review_cool_one)

In [ ]:
word_reviews_test = []
all_words = []
for review_test in reviews_processed_test:
    review_test = ViTokenizer.tokenize(review_test.lower())
    word_reviews_test.append(review_test.split())

print (word_reviews_test[0])